In [1]:
# importing sys
import sys
import os
import numpy as np
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/home/nfierroflo/Documents/mmdetection')
from inferencia import generate_mask,new_inference

torch version: 2.0.0+cu117 cuda: True
mmdetection: 3.0.0
mmcv: 2.0.0
mmengine: 0.7.2


In [2]:
#base
from save import *
name="2024-01-23-18-17-29_0"
redistribute_images(f"/media/nfierroflo/data/rosbags/{name}/left",f"/media/nfierroflo/data/rosbags/{name}/right",f"/home/nfierroflo/Documents/stereomatching/RAFT-Stereo/datasets/{name}")

In [3]:
#generate disparity maps
!source activate raftstereo && python demo.py --restore_ckpt models/raftstereo-middlebury.pth \
--corr_implementation alt --mixed_precision \
-l=datasets/2024-01-23-18-17-29_0/*/im0.jpg \
-r=datasets/2024-01-23-18-17-29_0/*/im1.jpg \
--save_numpy\
--output_directory demo_output/2024-01-23-18-17-29_0

/bin/bash: activate: No such file or directory


In [4]:
dataset_path="datasets/2024-01-23-18-17-29_0"
destination_path=f"demo_output/{name}/inferences"
os.makedirs(destination_path, exist_ok=True)
#iterate over in dataset_path
#show all the folders in dataset_path
#avoid WARNINGS messages

for testF_folder in tqdm(sorted(os.listdir(dataset_path))):
    path=dataset_path+"/"+testF_folder

    generated_mask=generate_mask(path+'/im0.jpg')

    indices=torch.where(generated_mask.pred_instances.labels==0)
    
    masks=generated_mask.pred_instances.masks[indices]
    
    inf_path=destination_path+"/"+testF_folder
    
    os.makedirs(inf_path, exist_ok=True)

    torch.save(masks.cpu(),inf_path+"/masks.pt")
    break


  0%|          | 0/763 [00:00<?, ?it/s]

Loads checkpoint by local backend from path: /home/nfierroflo/Documents/mmdetection/checkpoints/mask_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.38__segm_mAP-0.344_20200504_231812-0ebd1859.pth


  0%|          | 0/763 [00:02<?, ?it/s]


In [ ]:
for testF_folder in tqdm(sorted(os.listdir(dataset_path))):
    path=dataset_path+"/"+testF_folder

    masks,scores,labels=new_inference(path+'/im0.jpg')

    inf_path=destination_path+"/"+testF_folder
    
    os.makedirs(inf_path, exist_ok=True)

    np.save(inf_path+"/masks.npy",masks.cpu())
    np.save(inf_path+"/scores.npy",scores.cpu())
    np.save(inf_path+"/labels.npy",labels.cpu())

In [5]:
#withoutvelodyne
from detectionwithvelodyne import *

allPoints=[]
time=[]

for inference,image in tqdm(zip(sorted(os.listdir(destination_path)),sorted(os.listdir(dataset_path)))):
    images_folder_path = Path(dataset_path+"/"+image)
    inferences_folder_path= Path(destination_path+"/"+inference)
    disparity_path = Path(f"demo_output/{name}/{inference}.npy")
    velodynepoints_paths=[]
    
    save_folder_path=Path(f"demo_output/{name}/3ddetections/"+inference)
    os.makedirs(save_folder_path, exist_ok=True)
    save_path=os.path.join(save_folder_path,"/pointcloud.html")
    save_path=Path(f"demo_output/{name}/3ddetections/{inference}/{inference}withcentroids_person1.html")
    try:
        fig,P,points_data=visualize_pointcloud(images_folder_path,disparity_path,inferences_folder_path,velodynepoints_paths,save_path,use_velodyne=False,save_html=False,filter_label=True,objects_index=[3])
        time.append(float(inference))
        allPoints.append(points_data)
    except:
        pass

763it [08:33,  1.49it/s]


In [6]:
#get one vector for each value T,X,Y,Z
T=[]
X=[]
Y=[]
Z=[]
for i in range(len(allPoints)):
    if allPoints[i]!=[]:
        for j in range(len(allPoints[i][0][0])):
            T.append(time[i])
            X.append(allPoints[i][0][0][j])
            Y.append(allPoints[i][0][1][j])
            Z.append(allPoints[i][0][2][j])
    ##Decide how to deal with 0 detections
    else:
        T.append(-time[i])
        X.append(0)
        Y.append(0)
        Z.append(0)

In [7]:
import pandas as pd
df = pd.DataFrame(list(zip(T, X, Y, Z)),columns =['T', 'X', 'Y', 'Z'])
df.to_csv(f"demo_output/{name}/person4seq.csv",index=False,sep=',',header=False)

In [ ]:
import os
import numpy as np
import pandas as pd

# Function to extract timestamp from the file name
def get_timestamp_from_filename(filename):
    return float(filename.split("_")[-1][:-4])

# Folder path containing the .npy files for X, Y, and Z
velodynepoints_folder = "/media/nfierroflo/data/rosbags/2024-01-23-18-17-29_0/velodynevelodyne_points/"

# Load all .npy files and store them in lists for X, Y, and Z
X_files = [f for f in os.listdir(velodynepoints_folder) if f.startswith("X_")]
Y_files = [f for f in os.listdir(velodynepoints_folder) if f.startswith("Y_")]
Z_files = [f for f in os.listdir(velodynepoints_folder) if f.startswith("Z_")]

# Sort files by timestamp (assuming the timestamps are included in the filenames)
X_files.sort(key=get_timestamp_from_filename)
Y_files.sort(key=get_timestamp_from_filename)
Z_files.sort(key=get_timestamp_from_filename)

data = []
T=[]
X=[]
Y=[]
Z=[]
# Load data and timestamps for each timestamp
for X_file, Y_file, Z_file in zip(X_files, Y_files, Z_files):
    timestamp = get_timestamp_from_filename(X_file)
    X_data = np.load(os.path.join(velodynepoints_folder, X_file)).flatten()
    Y_data = np.load(os.path.join(velodynepoints_folder, Y_file)).flatten()
    Z_data = np.load(os.path.join(velodynepoints_folder, Z_file)).flatten()
    
    for x,y,z in zip(X_data,Y_data,Z_data):
        X.append(x)
        Y.append(y)
        Z.append(z)
        print(timestamp)
        T.append(timestamp)

#create a pandas dataframe
df = pd.DataFrame(list(zip(T, X, Y, Z)),columns =['T', 'X', 'Y', 'Z'])
print(df.head())
df.to_csv(f"demo_output/{name}/{name}_velodyne_seq.csv",index=False,sep=',',header=False)